In [7]:
import pandas as pd
import requests as rqs
import os
import json
import pymongo
import datetime
import pytz

myclient = pymongo.MongoClient(f'mongodb://{os.getenv("MONGO_URL","192.168.2.1")}:27017',username=os.getenv("MONGO_USERNAME","hterra"),password=os.getenv("MONGO_PASSWORD","terrac0t@"), unicode_decode_error_handler='ignore')
activitywatch_db = myclient["activitywatch"]
aw_watcher_window_db = myclient["activitywatch"]["aw-watcher-window"]

In [8]:

data = rqs.get(f'http://localhost:5600/api/0/export').json()
host = ''
for key in data['buckets'].keys(): 
    if 'aw-watcher-window' in key:
        host = key.replace('aw-watcher-window_','')
        aw_window = data['buckets'][key]
    if 'aw-watcher-afk' in key:
        aw_afk = data['buckets'][key]
events = [{
    "timestamp": e['timestamp'],
    "duration": e['duration'],
    "host": host.upper(),
    **e["data"]
} for e in aw_window['events']]



In [34]:
df = pd.DataFrame(events)
df['timestamp'] = pd.to_datetime(df['timestamp'],infer_datetime_format=True)
df['date'] = [x.astimezone(pytz.timezone('America/Mexico_City'))
             for x in df['timestamp']]
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute
df['time'] = df['duration'].apply(lambda x:
    '{:.0f} hr {:.0f} min {:.0f} secs'.format(
        divmod(x,60*60)[0],
        *divmod(divmod(x,60*60)[1],60))
                                 )
df = df.drop(['timestamp','date'], axis=1)

df

,duration,host,app,title,year,month,day,hour,minute,time
0,10.384,DESKTOP-RIEHMGE,chrome.exe,Untitled - Jupyter Notebook - Google Chrome,2020,3,12,8,51,0 hr 0 min 10 secs
1,28.320,DESKTOP-RIEHMGE,chrome.exe,How to read the exported JSON in pandas datafr...,2020,3,12,8,51,0 hr 0 min 28 secs
2,0.000,DESKTOP-RIEHMGE,chrome.exe,ActivityWatch - Google Chrome,2020,3,12,8,51,0 hr 0 min 0 secs
3,12.519,DESKTOP-RIEHMGE,chrome.exe,Untitled - Jupyter Notebook - Google Chrome,2020,3,12,8,50,0 hr 0 min 13 secs
4,59.488,DESKTOP-RIEHMGE,chrome.exe,How to read the exported JSON in pandas datafr...,2020,3,12,8,49,0 hr 0 min 59 secs
...,...,...,...,...,...,...,...,...,...,...
75794,1.074,DESKTOP-RIEHMGE,explorer.exe,Program Manager,2020,1,9,8,48,0 hr 0 min 1 secs
75795,1.066,DESKTOP-RIEHMGE,explorer.exe,,2020,1,9,8,48,0 hr 0 min 1 secs
75796,12.548,DESKTOP-RIEHMGE,ApplicationFrameHost.exe,Configuración,2020,1,9,8,48,0 hr 0 min 13 secs
75797,1.042,DESKTOP-RIEHMGE,explorer.exe,,2020,1,9,8,48,0 hr 0 min 1 secs


In [19]:
data_apps_global = df
data_apps_global['time'] = data_apps_global['timestamp'].dt.strftime('%H')
data_apps_global = data_apps_global.groupby(['time'], as_index=False)['duration'].sum()
data_apps_global['timestr'] = data_apps_global['duration'].apply(lambda x:
    '{:.0f} hr {:.0f} min {:.0f} secs'.format(
        divmod(x,60*60)[0],
        *divmod(divmod(x,60*60)[1],60))
                                 )
data_apps_global

,time,duration,timestr
0,00,196734.108,54 hr 38 min 54 secs
1,01,13054.434,3 hr 37 min 34 secs
2,02,4774.997,1 hr 19 min 35 secs
3,03,10153.710,2 hr 49 min 14 secs
4,04,83517.971,23 hr 11 min 58 secs
5,05,4471.264,1 hr 14 min 31 secs
6,06,918.531,0 hr 15 min 19 secs
7,07,8.288,0 hr 0 min 8 secs
8,08,2.340,0 hr 0 min 2 secs
9,14,68647.482,19 hr 4 min 7 secs


In [18]:
list(df['timestamp'])

[Timestamp('2020-03-12 14:51:42.231000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:51:12.859000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:51:11.794000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:50:58.233000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:49:57.689000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:49:51.358000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:49:47.132000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:49:41.908000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:49:27.253000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:49:20.946000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:49:15.676000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:49:07.270000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:46:29.407000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:46:24.120000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:45:33.843000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:45:32.788000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:45:31.720000+0000', tz='UTC'),
 Timestamp('2020-03-12 14:45:06

In [37]:
data_apps_global = df
data_apps_global.loc[data_apps_global['hour'].astype(int) == 23]

,duration,host,app,title,year,month,day,hour,minute,time
10452,6.919,DESKTOP-RIEHMGE,LockApp.exe,Pantalla de bloqueo predeterminada de Windows,2020,3,2,23,12,0 hr 0 min 7 secs
51426,921.321,DESKTOP-RIEHMGE,LockApp.exe,Pantalla de bloqueo predeterminada de Windows,2020,2,4,23,52,0 hr 15 min 21 secs
58571,1823.536,DESKTOP-RIEHMGE,LockApp.exe,Pantalla de bloqueo predeterminada de Windows,2020,1,24,23,16,0 hr 30 min 24 secs
58572,2.260,DESKTOP-RIEHMGE,unknown,,2020,1,24,23,16,0 hr 0 min 2 secs
58573,0.000,DESKTOP-RIEHMGE,unknown,,2020,1,24,23,16,0 hr 0 min 0 secs
...,...,...,...,...,...,...,...,...,...,...
65464,1.067,DESKTOP-RIEHMGE,chrome.exe,Software - Freeware applications - Google Chrome,2020,1,15,23,5,0 hr 0 min 1 secs
65465,0.000,DESKTOP-RIEHMGE,chrome.exe,CONTPAQi extraer claves de revocación y series...,2020,1,15,23,5,0 hr 0 min 0 secs
65466,0.000,DESKTOP-RIEHMGE,Teams.exe,Saved | Microsoft Teams,2020,1,15,23,5,0 hr 0 min 0 secs
65467,1.034,DESKTOP-RIEHMGE,chrome.exe,CONTPAQi extraer claves de revocación y series...,2020,1,15,23,5,0 hr 0 min 1 secs


In [43]:
import json
records = json.loads(df.T.to_json()).values()
activitywatch_db['test'].insert_many(records)


In [51]:
#df = pd.DataFrame(list(aw_watcher_window_db.find()))
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['time'] = df['duration'].apply(lambda x:
    '{:.0f} hr {:.0f} min {:.0f} secs'.format(
        divmod(x,60*60)[0],
        *divmod(divmod(x,60*60)[1],60))
                                 )
df = df.drop(['timestamp'], axis=1)



aw_watcher_window_db.delete_many({})


OverflowError: Unsupported UTF-8 sequence length when encoding string

In [52]:
df


,_id,duration,host,app,title,year,month,day,hour,minute,time
0,5e6805f80be82013dd33a0fd,97.264,DESKTOP-RIEHMGE,unknown,,2020,3,10,21,23,0 hr 1 min 37 secs
1,5e6805f80be82013dd33a0fe,300.523,DESKTOP-RIEHMGE,chrome.exe,Untitled - Jupyter Notebook - Google Chrome,2020,3,10,21,18,0 hr 5 min 1 secs
2,5e6805f80be82013dd33a0ff,18.623,DESKTOP-RIEHMGE,MongoDBCompass.exe,MongoDB Compass - 192.168.2.1:27017/activitywa...,2020,3,10,21,17,0 hr 0 min 19 secs
3,5e6805f80be82013dd33a100,0.000,DESKTOP-RIEHMGE,MongoDBCompass.exe,MongoDB Compass - 192.168.2.1:27017/activitywa...,2020,3,10,21,17,0 hr 0 min 0 secs
4,5e6805f80be82013dd33a101,15.291,DESKTOP-RIEHMGE,chrome.exe,Untitled - Jupyter Notebook - Google Chrome,2020,3,10,21,17,0 hr 0 min 15 secs
...,...,...,...,...,...,...,...,...,...,...,...
706329,5e6819ac6c984a5fa412e52c,4.274,DESKTOP-PDQAQA7,SearchUI.exe,Búsqueda,2020,1,24,18,18,0 hr 0 min 4 secs
706330,5e6819ac6c984a5fa412e52d,3.457,DESKTOP-PDQAQA7,StartMenuExperienceHost.exe,Inicio,2020,1,24,18,18,0 hr 0 min 3 secs
706331,5e6819ac6c984a5fa412e52e,1.238,DESKTOP-PDQAQA7,SearchUI.exe,Búsqueda,2020,1,24,18,18,0 hr 0 min 1 secs
706332,5e6819ac6c984a5fa412e52f,2.817,DESKTOP-PDQAQA7,StartMenuExperienceHost.exe,Inicio,2020,1,24,18,18,0 hr 0 min 3 secs


In [55]:
aw_watcher_window_db.insert_many( df.to_dict(orient='records'))

In [54]:
data =  df.to_dict(orient='records')

[{'_id': ObjectId('5e6805f80be82013dd33a0fd'),
  'duration': 97.264,
  'host': 'DESKTOP-RIEHMGE',
  'app': 'unknown',
  'title': '',
  'year': 2020,
  'month': 3,
  'day': 10,
  'hour': 21,
  'minute': 23,
  'time': '0 hr 1 min 37 secs'},
 {'_id': ObjectId('5e6805f80be82013dd33a0fe'),
  'duration': 300.523,
  'host': 'DESKTOP-RIEHMGE',
  'app': 'chrome.exe',
  'title': 'Untitled - Jupyter Notebook - Google Chrome',
  'year': 2020,
  'month': 3,
  'day': 10,
  'hour': 21,
  'minute': 18,
  'time': '0 hr 5 min 1 secs'},
 {'_id': ObjectId('5e6805f80be82013dd33a0ff'),
  'duration': 18.623,
  'host': 'DESKTOP-RIEHMGE',
  'app': 'MongoDBCompass.exe',
  'title': 'MongoDB Compass - 192.168.2.1:27017/activitywatch.aw-watcher-window',
  'year': 2020,
  'month': 3,
  'day': 10,
  'hour': 21,
  'minute': 17,
  'time': '0 hr 0 min 19 secs'},
 {'_id': ObjectId('5e6805f80be82013dd33a100'),
  'duration': 0.0,
  'host': 'DESKTOP-RIEHMGE',
  'app': 'MongoDBCompass.exe',
  'title': 'MongoDB Compass - 192

In [56]:
from time import gmtime, strftime
print(strftime("%z", gmtime()))

-0600
